In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
img1 = cv2.imread("1.jpg")
img2 = cv2.imread("2.jpg")
img3 = cv2.imread("3.jpg")
img4 = cv2.imread("4.jpg")
img5 = cv2.imread("5.jpg")
img6 = cv2.imread("6.jpg")
images = [img1, img2, img3, img4, img5, img6]

In [41]:
stitcher = cv2.Stitcher_create()

# Stitch horizontally
result_horizontal, stitched_horizontal = stitcher.stitch((img1, img2, img3))

# Stitch vertically
result_vertical, stitched_vertical = stitcher.stitch((img3, img5, img6))

# Check if stitching was successful
if result_horizontal == cv2.Stitcher_OK:
    cv2.imwrite('stitched_horizontal.jpg', stitched_horizontal)
    cv2.imshow('Stitched Horizontal', stitched_horizontal)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Horizontal stitching failed.")

if result_vertical == cv2.Stitcher_OK:
    cv2.imwrite('stitched_vertical.jpg', stitched_vertical)
    cv2.imshow('Stitched Vertical', stitched_vertical)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Vertical stitching failed.")

Horizontal stitching failed.
Vertical stitching failed.


In [9]:
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
gray3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
gray4 = cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY)
gray5 = cv2.cvtColor(img5, cv2.COLOR_BGR2GRAY)
gray6 = cv2.cvtColor(img6, cv2.COLOR_BGR2GRAY)

# Initialize the feature detector (SIFT or ORB can be used)
detector = cv2.ORB_create()

# Detect keypoints and compute descriptors
kp1, des1 = detector.detectAndCompute(gray1, None)
kp2, des2 = detector.detectAndCompute(gray2, None)
kp3, des3 = detector.detectAndCompute(gray3, None)
kp4, des4 = detector.detectAndCompute(gray4, None)
kp5, des5 = detector.detectAndCompute(gray5, None)
kp6, des6 = detector.detectAndCompute(gray6, None)

# Initialize the feature matcher
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors between adjacent images
matches1to2 = matcher.match(des1, des2)
matches4to5 = matcher.match(des4, des5)

# Sort matches by distance
matches1to2 = sorted(matches1to2, key=lambda x: x.distance)
matches4to5 = sorted(matches4to5, key=lambda x: x.distance)

# Extract matched keypoints
matched_kp1to2 = np.float32([kp1[m.queryIdx].pt for m in matches1to2]).reshape(-1, 1, 2)
matched_kp2to1 = np.float32([kp2[m.trainIdx].pt for m in matches1to2]).reshape(-1, 1, 2)

matched_kp4to5 = np.float32([kp4[m.queryIdx].pt for m in matches4to5]).reshape(-1, 1, 2)
matched_kp5to4 = np.float32([kp5[m.trainIdx].pt for m in matches4to5]).reshape(-1, 1, 2)

# Estimate homography
H1to2, _ = cv2.findHomography(matched_kp1to2, matched_kp2to1, cv2.RANSAC, 5.0)
H4to5, _ = cv2.findHomography(matched_kp4to5, matched_kp5to4, cv2.RANSAC, 5.0)

# Warp images
result1to2 = cv2.warpPerspective(img1, H1to2, (img2.shape[1] + img1.shape[1], img2.shape[0]))
result1to2[0:img2.shape[0], 0:img2.shape[1]] = img2

result4to5 = cv2.warpPerspective(img4, H4to5, (img5.shape[1] + img4.shape[1], img5.shape[0]))
result4to5[0:img5.shape[0], 0:img5.shape[1]] = img5

# Display results
cv2.imshow('Result 1 to 2', result1to2)
cv2.imshow('Result 4 to 5', result4to5)
cv2.imwrite("output12.jpg", result1to2)
cv2.imwrite("output45.jpg", result4to5)
cv2.waitKey(0)
cv2.destroyAllWindows()